# Funciones y librerias

In [1]:
import cv2                                  # Biblioteca para Tareas de Vision por Computadora.            
import numpy as np                          # Biblioteca para Tareas de Calculo Numerico.                  
from matplotlib import pyplot as plt        # Biblioteca para Tareas de Visualizacion de Datos y Graficos. 
import copy                                 # Biblioteca para realizar operaciones de copiado de objetos.    
from PIL import ImageGrab,Image                  # Biblioteca para capturar imágenes de la pantalla (screen capture) utilizando la librería Pillow.
import os                                   # Biblioteca para interactuar con el sistema operativo, incluyendo operaciones con archivos y directorios.
import time                                 # Biblioteca para trabajar con funciones relacionadas con el tiempo, como pausas y temporizadores.
import keyboard                             # Biblioteca para detectar y manejar eventos de teclado, como la detección de pulsaciones de teclas.
import serial                               # Importo la librería serial para manejar la comunicación serial

########################################################################################################################################################
########################################################################################################################################################
#                                                     FUNCIONES PARA USAR EN CHESS.COM                                                                 #
########################################################################################################################################################
########################################################################################################################################################

def deteccion (lower_h, lower_s, lower_v, upper_h, upper_s, upper_v, ultima_imagen_capturada):  # Como entrada ingresando los valores de los colores que deseo detectar en hsv lower y upper , tambien la entrada es la ruta de una imagen
    #--------------------------------[Procesamiento de la imagen recortada]-----------------------------------#

    image =  cv2.imread(ultima_imagen_capturada)                                            # Leo la imagen desde la ruta especificada por 'ultima_imagen_capturada'.
    imagen_copia = image.copy()                                                             # Hago una copia de la imagen original para evitar modificarla directamente.

    Porcentaje_de_reduccion = 0.4                                                           # Tendre el 40% de la imagen original, es decir que si era de 100px, ahora tendre 40px
    imagen_redimensionada = cv2.resize(imagen_copia, (0, 0), fx=Porcentaje_de_reduccion, fy=Porcentaje_de_reduccion)   # Reducimos el tamaño de laimagen, para evitar errores de deteccion de color, y que sea mas facil obtener los contornos de las piezas

    hsv_image = cv2.cvtColor(imagen_redimensionada, cv2.COLOR_BGR2HSV)                      # Convierto la imagen del espacio de color BGR (Azul, Verde, Rojo) a HSV (Hue, Saturation, Value). El espacio de color HSV separa la información de color (tono), la saturación (intensidad del color) y el valor (brillo) de manera más efectiva que el BGR

    lower_red = np.array([lower_h, lower_s, lower_v])                                       # Defino el rango de colores que quiero detectar en la imagen en formato HSV (Debe ajustarse segun sea necesario) . lower_red define el valor mínimo (Hue, Saturation, Value) y upper_red define el valor máximo para la detección                       
    upper_red = np.array([upper_h, upper_s, upper_v])                                       # Defino el rango de colores que quiero detectar en la imagen en formato HSV (Debe ajustarse segun sea necesario) . lower_red define el valor mínimo (Hue, Saturation, Value) y upper_red define el valor máximo para la detección                       

    mask = cv2.inRange(hsv_image, lower_red, upper_red)                                     # Creo una máscara binaria utilizando el rango de colores definido anteriormente.La mascara mostrara en blanco (255) lo que esta en el rango de colores y negro (0) lo que no
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)        # Busco los contornos de los objetos detectados en la máscara binaria.cv2.RETR_EXTERNAL devuelve solo los contornos externos (los bordes de los objetos), mientras que cv2.CHAIN_APPROX_SIMPLE comprime los contornos, eliminando los puntos redundantes y ahorrando memoria.
    nombre_variable = []                                                                    # Inicializo un vector para almacenar las coordenadas de los centros de los contornos detectados.

    #-------------------------[Graficas del procesamiento de la imagen recortada]------------------------------#

    #plt.figure(figsize=(10, 10))                                                           # Establecemos el tamaño de la figura en la que se mostrarán las imágenes (Pulgadas)

    #plt.subplot(2, 2, 1)                                                                   # Creamos un subplot en la primera fila y primera columna.
    #plt.imshow(hsv_image)                                                                  # Mostramos la imagen del espacio HSV
    #plt.title('Imagen en espacio de color HSV')                                            # Establecemos el título del subplot

    #plt.subplot(2, 2, 2)                                                                   # Creamos un subplot en la primera fila y segunda columna
    #plt.imshow(mask, cmap='gray')                                                          # Mostramos la imagen de la mascara binaria
    #plt.title('Máscara binaria')                                                           # Establecemos el título del subplot

    masked_image = cv2.bitwise_and(imagen_redimensionada, imagen_redimensionada, mask=mask)                   # cv2.bitwise_and() combina dos imágenes utilizando una máscara para determinar qué píxeles se mantienen y cuáles se descartan en la imagen resultante
    #plt.subplot(2, 2, 3)                                                                   # Creamos un subplot en la segunda fila y primera columna.
    #plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))                              # Mostramos la imagen original con la mascara,"cv2-cvtcolor" convierte la imagen masked_image del espacio de color BGR (Blue, Green, Red) al espacio de color RGB (Red, Green, Blue)
    #plt.title('Imagen original con la máscara binaria aplicada')                           # Establecemos el título del subplot

    contour_image = cv2.drawContours(masked_image.copy(), contours, -1, (255, 255, 255), 1) # cv2.drawContours() modifica la imagen sobre la que se dibujan los contornos. Al crear una copia de la imagen original, se conserva la imagen original intacta
    #plt.subplot(2, 2, 4)                                                                   # Creamos un subplot en la segunda fila y segunda columna
    #plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))                             # Mostramos la imagen original con contornos
    #plt.title('Imagen con contornos')                                                      # Establecemos el título del subplot

    #plt.tight_layout()                                                                     # Ajustamos automáticamente la disposición de los subplots para evitar superposiciones y mejorando la visualizacion
    #plt.show()                                                                             # Mostramos la figura con todos los subplots

    #-------------------------[Obtencion de coordenadas de las esquinas del tablero]------------------------------#

    for contour in contours:                                                                # Recorro cada contorno encontrado para calcular su centro y dibujar un círculo en ese punto.                                              
        moments = cv2.moments(contour)                                                      # Calculo los momentos de masa del contorno para encontrar su centro
        if moments["m00"] > 250:                                                            # Compruebo que el área del contorno no sea cero para evitar divisiones por cero.
            cx = int(moments["m10"] / moments["m00"])                                       # Coordenada x del centro del contorno.
            cy = int(moments["m01"] / moments["m00"])                                       # Coordenada y del centro del contorno.
            nombre_variable.append((cx, cy))                                                # Añado las coordenadas del centro al vector de coordenadas.

            cv2.circle(imagen_redimensionada, (cx, cy), 5, (0, 0, 255), -1)                                                                  # Se dibuja un círculo en la imagen original image en las coordenadas del centro del contorno. El círculo tiene un radio de 5 píxeles y está coloreado en verde (0, 255, 0).
            cv2.putText(imagen_redimensionada, f'({cx}, {cy})', (cx - 30 , cy -10 ), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)        # Se añade un texto con las coordenadas (cx, cy) del centro del contorno. El texto se posiciona en las coordenadas (cx - 30, cy - 10) con una fuente cv2.FONT_HERSHEY_SIMPLEX, tamaño de fuente 0.4, color blanco (255, 255, 255) y grosor 2.

    #print("Vector de coordenadas:", nombre_variable)
    print("\nCantidad de coordenadas:",len(nombre_variable))
    #plt.figure(figsize=(6, 6))                                                             # Establecemos el tamaño de la figura en la que se mostrarán las imágen(Pulgadas)
    #plt.title('Imagen con coordenadas resultante')                                         # Establecemos el título de plot
    #plt.imshow(cv2.cvtColor(imagen_redimensionada, cv2.COLOR_BGR2RGB))                              # Mostramos la imagen original con las coordenadas resultantes
    #plt.show()                                                                             # Mostramos la figura 
    
    return  nombre_variable                                                                 # Coordenadas de todas las piezas

# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#

def Armado_de_nodos_de_las_casillas(coordenadas_esquinas, ultima_imagen_capturada):         # Como entrada van las coordenadas de las 4 esquinas del tablero de ajedrez, y una imagen 
    #--------------------------------------[Division de casillas]---------------------------------------------#

    image = cv2.imread(ultima_imagen_capturada)                                             # Cargar la imagen del tablero de ajedrez nueva sin dibujos
    imagen_copia = image.copy()

    Porcentaje_de_reduccion = 0.7       # Se reducira un 70% Del tamñao original
    imagen_redimensionada = cv2.resize(imagen_copia, (0, 0), fx=Porcentaje_de_reduccion, fy=Porcentaje_de_reduccion)
    #print("Vector de coordenadas de las esquinas del tablero:", coordenadas)               # Ejemplo : Vector de coordenadas de las esquinas del tablero: [(9, 345), (350, 340), (346, 15), (12, 9)]

    x1 = coordenadas_esquinas[0][0]                                                         # Obtenemos la coordenada x de la esquina inferior izquierda
    x2 = coordenadas_esquinas[1][0]                                                         # Obtenemos la coordenada x de la esquina inferior derecha
    y1 = coordenadas_esquinas[0][1]                                                         # Obtenemos la coordenada y de la esquina inferior izquierda
    y4 = coordenadas_esquinas[3][1]                                                         # Obtenemos la coordenada y de la esquina superior izquierda

    dif_horizontal = np.abs(x2 - x1)                                                        # Ejemplo : |(350)-(9)| --> 341     # Calculamos la diferencia horizontal y vertical entre las esquinas
    dif_vertical = np.abs(y4 - y1)                                                          # Ejemplo : |(9)-(345)| --> 336

    segmento_horizontal = dif_horizontal // 8                                               # Dividimos las diferencias entre 8 para obtener el tamaño de cada casilla del tablero (RECIBO UN NUMERO ENTERO)
    segmento_vertical = dif_vertical // 8
    #print("Segmento horizontal:",segmento_horizontal)
    #print("Segmento vertical:",segmento_vertical)
    coordenadas_matriz = np.zeros((9, 9, 2), dtype=int)                                     # Creamos una matriz llena de ceros de 9 filas , 9 columnas y 2 profundidades (x,y). Los elementos de la matriz son enteros

    #-----------------------------[Imagen con puntos de la division de casillas]-------------------------------#

    for x in range(9):                                                                      # itero 9 veces, representando las divisiones horizontales del tablero.
        for y in range(9):                                                                  # itero 9 veces, representando las divisiones verticales del tablero.
            x_coord = int(x1 + x * segmento_horizontal)                                     # Se suma la coordenada de la esquina inferior izquierda (x1) con el producto del índice actual del bucle x y el segmento horizontal. El resultado se convierte a entero.
            y_coord = int(y1 - y * segmento_vertical)                                       # Se resta la coordenada de la esquina inferior izquierda (y1) con el producto del índice actual del bucle y y el segmento vertical.Se usa el signo negativo para ajustar la dirección hacia arriba. El resultado se convierte en entero  
            coordenadas_matriz[x, y] = [x_coord, y_coord]                                   # Agregar las coordenadas al vector
            cv2.circle(imagen_redimensionada, (x_coord, y_coord), 10, (0, 255, 0), -1)               # Dibujamos un punto en la(imagen_copia) en las coordenadas (x_coord, y_coord). El punto tiene un radio de 3 píxeles, color verde y el -1 indica que se rellena el circulo.

    #plt.figure(figsize=(6, 6))                                                             # Establecemos el tamaño de la figura en la que se mostrará la imágen (Pulgadas)
    #plt.title('Tablero de ajedrez con coordenadas 9x9')                                    # Establecemos el título de plot
    #plt.imshow(cv2.cvtColor(imagen_copia, cv2.COLOR_BGR2RGB))                              # Mostramos la imagen original (pero copia) con las coordenadas resultantes
    #plt.show()                                                                             # Mostramos la figura 

    MatrizTraspuesta = coordenadas_matriz.transpose(1,0,2)                                  # transpose() permuta las dimensiones de un array. (1,0,2) la segunda dimension se mueva a la primera posicion, la primera dimension se mueve a la segunda posicion, y hay 2 profundidades (x,y)                         

    #print("Matriz completa de cada coordenada del tablero:\n\n",MatrizTraspuesta.flatten(),"\n") # numpy.ndarray.flatten() es para aplanar la matriz y luego imprimir los elementos uno al lado del otro. 
    #print("fila 1:\n\n", coordenadas_matriz.transpose(1,0,2) [0,:])                        # Por si quisiera ver alguna fila o columna en especifico
    #print("columna 1:\n\n", coordenadas_matriz.transpose(1,0,2) [:,0])

    return MatrizTraspuesta,segmento_horizontal,segmento_vertical

# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#

def convertir_y_mostrar_resultados(nombre_variable, segmento_horizontal, segmento_vertical):    # Como entrada recibe las coordenadas de todas las piezas (en pixeles) , y las divisiones de cada celda

    casillas = []                                                                               # Inicializo una lista vacía para almacenar las coordenadas convertidas de las casillas del tablero.
    for coord in nombre_variable:                                                               # Recorro cada coordenada en la lista 'nombre_variable'.
        x, y = coord                                                                            # Descompongo la coordenada en las variables 'x' (posición horizontal) y 'y' (posición vertical).
        fila = y // segmento_horizontal                                                         # Calculo el número de fila dividiendo la coordenada vertical 'y' por el tamaño del segmento horizontal (división entera).
        columna = x // segmento_vertical                                                        # Calculo el número de columna dividiendo la coordenada horizontal 'x' por el tamaño del segmento vertical (división entera).
        casillas.append((fila, columna))                                                        # Añado la casilla calculada (fila, columna) a la lista 'casillas'.
        #print(f"Coordenada ({x}, {y}) -> Casilla ({fila}, {columna}) del tablero")             # (Opcional) Imprimo la coordenada original y la casilla correspondiente en el tablero para verificar los resultados.
    return casillas                                                                             # Devuelvo la lista de casillas calculadas (en fila,columna).

# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#

def detectar_enroque(casillas_anteriores, casillas_actuales):                               # Como entrada, se reciben unicamente las casillas de las piezas que esten jugando actualmente.

    set_anteriores = set(casillas_anteriores)                                               # Convierto las coordenadas de las casillas anteriores en un conjunto para facilitar la comparación.                                 
    set_actuales = set(casillas_actuales)                                                   # Convierto las coordenadas de las casillas actuales en un conjunto para facilitar la comparación.
    
    diferencias = set_anteriores - set_actuales                                             # Encuentro las diferencias entre las casillas anteriores y las actuales.
    #print(f"Diferencias entre las coordenadas: {diferencias}")                             # Imprimir las diferencias entre las casillas (Opcional)
    
    if len(diferencias) == 2:                                                               # Si las casillas de las diferencias son exacatamente igual a 2, significa que hubo un enroque y entro al if
        
        diferencias_lista = sorted(diferencias)                                             # Convierto el conjunto de diferencias en una lista y la ordeno para facilitar la comparación.

        if diferencias_lista == [(7, 4), (7, 7)] or diferencias_lista == [(7, 7), (7, 4)]:  # Verifico si las diferencias corresponden a un enroque corto de las blancas.
            print("Se produjo un enroque corto de las blancas")
            enroque = 4                                                                     # Si el enroque es corto y de las blancas, seteo la variable enroque en 4
        elif diferencias_lista == [(7, 4), (7, 0)] or diferencias_lista == [(7, 0), (7, 4)]:# Verifico si las diferencias corresponden a un enroque largo de las blancas.
            print("Se produjo un enroque largo de las blancas")
            enroque = 5                                                                     # Si el enroque es largo y de las blancas, seteo la variable enroque en 5
        elif diferencias_lista == [(0, 4), (0, 7)] or diferencias_lista == [(0, 7), (0, 4)]:# Verifico si las diferencias corresponden a un enroque corto de las negras.
            print("Se produjo un enroque corto de las negras")
            enroque = 2                                                                     # Si el enroque es corto y de las negras, seteo la variable enroque en 2 
        elif diferencias_lista == [(0, 4), (0, 0)] or diferencias_lista == [(0, 0), (0, 4)]:# Verifico si las diferencias corresponden a un enroque largo de las negras.
            print("Se produjo un enroque largo de las negras")
            enroque = 3                                                                     # Si el enroque es largo y de las negras, seteo la variable enroque en 3 
    else:                                                                                   # Si no se detectan exactamente dos diferencias, no hay enroque.
        #print("No hay enroque")
        enroque = 6                                                                         # Para el caso de que no haya ningun enroque, seteo la variable enroque en 6s
    
    return enroque                                                                          # Retorno 'enroque' que indica el tipo de enroque detectado o si no se detectó ninguno.

# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#

def determinar_cambio_de_piezas(casillas_anteriores, casillas_actuales, casillas_oponentes_anteriores, casillas_oponentes_actuales):

    origen = None                                                                           # Inicializo la variable 'origen' para almacenar la casilla de origen donde se ha producido el movimiento de una pieza.
    destino = None                                                                          # Inicializo la variable 'destino' para almacenar la casilla de destino donde se ha movido una pieza
    pieza_comida = False                                                                    # Inicializo 'pieza_comida' como False para indicar si se ha capturado una pieza oponente durante el movimiento.
    
    enroque = detectar_enroque(casillas_anteriores, casillas_actuales)                      # LLamo a la funcion para detectar enroque, y saber que valor tiene enroque.

    if enroque != 6:                                                                        # Solo entra si el enroque es diferente de '6'
        if enroque == 2:                                                                    # Enroque corto negro
            origen = (0,4)                                                                  # Rey negro coordenada origen
            destino = (0,6)                                                                 # Rey negro coordenada destino
        elif enroque == 3:                                                                  # Enroque largo negro 
            origen = (0,4)                                                                  # Rey negro coordenada origen
            destino = (0,2)                                                                 # Rey negro coordenada destino
        elif enroque == 4:                                                                  # Enroque corto blanco 
            origen = (7,4)                                                                  # Rey blanco origen
            destino = (7,6)                                                                 # Rey blanco destino
        elif enroque == 5:                                                                  # Enroque largo blanco 
            origen = (7,4)                                                                  # Rey blanco origen
            destino = (7,2)                                                                 # Rey blanco destino
    else:                                                                                   # Sino si... Enroque es igual a '6' (condicion de que no hay enroque)
        set_anteriores = set(casillas_anteriores)                                           # Convierte la lista de casillas anteriores a un conjunto para facilitar la comparación con las casillas actuales
        set_actuales = set(casillas_actuales)                                               # Convierte la lista de casillas actuales a un conjunto para facilitar la comparación con las casillas anteriores.
        set_oponentes_anteriores = set(casillas_oponentes_anteriores)                       # Convierte la lista de casillas del oponente anteriores a un conjunto para facilitar la comparación con las casillas actuales del oponente.
        set_oponentes_actuales = set(casillas_oponentes_actuales)                           # Convierte la lista de casillas actuales del oponente a un conjunto para facilitar la comparación con las casillas anteriores del oponente.

        casilla_vaciada = list(set_anteriores - set_actuales)                               # Encuentra las casillas que estaban en el conjunto de casillas anteriores pero no en el de casillas actuales. Esta es la casilla que se ha vaciado.
        if casilla_vaciada:                                                                 # Verifica si se ha encontrado una casilla vaciada.
            origen = casilla_vaciada[0]                                                     # Asigna la primera casilla vaciada encontrada a la variable 'origen'. (Basicamente esto sirve para encontrar la casilla de origen (la que se vacio))

        casilla_ocupada = list(set_actuales - set_anteriores)                               # Encuentra las casillas que están en el conjunto de casillas actuales pero no en el de casillas anteriores. Esta es la casilla que se ha ocupado.
        if casilla_ocupada:                                                                 # Verifica si se ha encontrado una casilla ocupada.
            destino = casilla_ocupada[0]                                                    # Asigna la primera casilla ocupada encontrada a la variable 'destino'. (Basicamente esto sirve para encontrar la casilla de destino (la que se ocupa))

        if destino in set_oponentes_anteriores and destino not in set_oponentes_actuales:   # Verifica si la casilla de destino estaba ocupada por una pieza del oponente anteriormente pero ahora está vacía en las casillas actuales del oponente.
            pieza_comida = True                                                             # Si la pieza oponente ha sido comida, establece 'pieza_comida' a True. Caso contrario sera False

    return origen, destino, pieza_comida, enroque                                           # Devuelve el tablero actualizado despues del movimiento, y un vector de 3 posiciones con las coordenadas reales del movimiento en "str"

# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#

def mover_pieza(tablero, origen, destino, enroque):                                         # Recibo como entrada el tablero actual (lista), la coordenada de origen (fila,columna), la coordenada de destino (fila,columna) y la variable de enroque
    
    columnas = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']                                     # Lista que representa las columnas del tablero de ajedrez en notación algebraica.

    if enroque != 6:                                                                        # Si hubo enroque, entro al if.
        if enroque == 2:                                                                    # Enroque corto de las negras
            tablero[0][5] = 'r'                                                             # Mover torre a la posición correcta
            tablero[0][7] = ' '                                                             # Vaciar la posición de la torre vieja
        elif enroque == 3:                                                                  # Enroque largo de las negras
            tablero[0][3] = 'r'                                                             # Mover torre a la posición correcta
            tablero[0][0] = ' '                                                             # Vaciar la posición de la torre vieja
        elif enroque == 4:                                                                  # Enroque corto de las blancas
            tablero[7][5] = 'R'                                                             # Mover torre a la posición correcta
            tablero[7][7] = ' '                                                             # Vaciar la posición de la torre vieja
        elif enroque == 5:                                                                  # Enroque largo de las blancas
            tablero[7][3] = 'R'                                                             # Mover torre a la posición correcta
            tablero[7][0] = ' '                                                             # Vaciar la posición de la torre vieja

    pieza = tablero[origen[0]][origen[1]]                                                   # Extraigo la pieza ubicada en las coordenadas de origen del tablero.
    tablero[origen[0]][origen[1]] = ' '                                                     # Coloco un espacio vacío en la posición de origen del tablero, indicando que la pieza ha sido movida.
    pieza_comida = tablero[destino[0]][destino[1]]                                          # Guardamos la pieza que estaba en el destino (si es que había alguna) antes de mover la nueva pieza a esa posición.                                       
    tablero[destino[0]][destino[1]] = pieza                                                 # Coloco la pieza extraída en la nueva posición de destino en el tablero.
        
    origen_real = f"{columnas[origen[1]]}{8-origen[0]}"                                     # Convierte las coordenadas de origen (fila, columna) a la notación algebraica (letra-columna y número-fila).
    destino_real = f"{columnas[destino[1]]}{8-destino[0]}"                                  # Convierte las coordenadas de destino (fila, columna) a la notación algebraica (letra-columna y número-fila).
    
    if enroque != 6:                                                                        # Compruebo si se hizo un enroque
        vector_movimiento = [origen_real, destino_real, str(enroque)]                       # El vector de movimeinto incluye el valor de la variable 'enroque' (Que puede ser : '2','3','4','5') para indicar el tipo de enroque.
    elif pieza_comida != ' ':                                                               # Comprueba si había una pieza en la posición de destino antes del movimiento.
        vector_movimiento = [origen_real, destino_real, '1']                                # Si había una pieza comida,y tampoco hubo enroque, el vector de movimiento incluye '1' para indicar que se ha capturado una pieza.
    else:                                                                                   
        vector_movimiento = [origen_real, destino_real, '0']                                # Si no había pieza comida,y tampoco hubo enroque, el vector de movimiento incluye '0' para indicar que no se ha capturado ninguna pieza.
        
    print(f"Moviendo la pieza de {origen_real} a {destino_real}")                           # Imprime en consola la acción de movimiento en notación algebraica para visualización, desde el origen hasta el destino.
        
    return tablero, vector_movimiento                                                       # Devuelve el tablero actualizado despues del movimiento, y un vector de 3 posiciones con las coordenadas reales del movimiento en "str"

# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#

arduino = serial.Serial('COM4', 9600, timeout=1)                                            # Abro el puerto 'COM4' con una velocidad de 9600 baudios y un tiempo de espera de 1 segundo (Reemplaza 'COM X' con el puerto adecuado)


def enviar_coordenadas_a_arduino(coordenadas):                                              # Funcion para mandar el 'vector de movimeinto' con formato de ejemplo ['G1', 'F3', '0']
    if len(coordenadas) == 3:                                                               # Verifico si el vector coordenadas tiene exactamente 3 elementos (inicial, final, es_comer).
        coord_inicial = coordenadas[0]                                                      # Extraigo la coordenada inicial del 'vector de movimientos'.
        coord_final = coordenadas[1]                                                        # Extraigo la coordenada final del 'vector de movimientos'.
        es_comer = coordenadas[2]                                                           # Extraigo el indicador de captura de pieza del 'vector de movimientos'.
        mensaje = f"{coord_inicial} {coord_final} {es_comer}\n"                             # Formato el mensaje para enviarlo al Arduino, separando los elementos con un espacio y Agrego un salto de línea al final del mensaje (importante esto ultimo debe estar si o si para que funcione con mis funciones de arduino)
        arduino.write(mensaje.encode())                                                     # Envio el mensaje codificado en formato de bytes al puerto serial
        print(f"\nEnviando coordenadas: {mensaje}")                                         # Imprimo el mensaje enviado para verificar que se envió correctamente
    else:
        print("Se necesitan exactamente dos coordenadas válidas para enviar.")              # Si el vector no tiene exactamente 3 elementos, imprimo un mensaje de error


# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#

def capture_screenshots():                                                                                  # Funcion del bucle general, no tiene entradas ni salidas, pero si esta la parte mas importante del codigo.

    Tablero_inicial =  [['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'],                                           # Definir el tablero inicial de ajedrez 
                        ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
                        [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
                        [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
                        [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
                        [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
                        ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
                        ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']]
    
    coords = (332, 116, 1180, 965)                                                                              # Defino las coordenadas de recorte para capturar la pantalla (Las obtuve con una pagina:https://www.image-map.net/)
    save_dir = r"C:\Users\gasto\OneDrive\Escritorio\Codigos\Python\Codigo de chess.com\Capturas de pantalla"    # Defino el directorio donde se guardarán las capturas de pantalla
    
    os.makedirs(save_dir, exist_ok=True)                                                                    # Crear el directorio si no existe

    capture_count = 0                                                                                       # Inicializo el contador de capturas en 0

    lower_hsv = np.array([28, 0, 0])                                                                        # Defino el rango de colores en HSV (De las piezas blancas y negras:[1, 0, 0][40, 255, 255])
    upper_hsv = np.array([180, 255, 255])
    lower_hsv_white = np.array([21, 0, 0])                                                                  # Defino el rango de colores en HSV (Solo de las piezas blancas:[18, 0, 0][180, 255, 255])
    upper_hsv_white = np.array([180, 85, 255])
    lower_hsv_black = np.array([35, 35, 0])                                                                 # Defino el rango de colores en HSV (Solo de las piezas negras: [2, 40, 0][180, 255, 255]) 
    upper_hsv_black = np.array([180, 255, 255])                                         
    
    screenshot_prev = ImageGrab.grab(bbox=coords)                                                           # Capturar el primer frame
    
    casillas_anterior_blanca = [(7, 7), (7, 0), (7, 5), (7, 4), (7, 2), (7, 3), (7, 6), (7, 1), (6, 7), (6, 5), (6, 3), (6, 1), (6, 6), (6, 4), (6, 2), (6, 0)]     # Inicializamos listas de casillas blancas de la iteración anterior, osea las iniciales al estar fuera del while
    casillas_anterior_negra =  [(1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (0, 7), (0, 0), (0, 5), (0, 4), (0, 2), (0, 6), (0, 1), (0, 3)]     # Inicializamos listas de casillas negras de la iteración anterior, osea las iniciales al estar fuera del while

    tablero_actual = [fila[:] for fila in Tablero_inicial]                                                  # Inicializamo el tablero actual con el tablero inicial
    turno_blancas = True                                                                                    # Empezamos siempre con el turno de las blancas

    while True:                                                                                             # Bucle principal e infinito para el codigo de chess.com
        screenshot = ImageGrab.grab(bbox=coords)                                                            # Se captura la pantalla dentro de las coordenadas especificadas en 'coords'

        screenshot_np = np.array(screenshot)                                                                # Convierto la captura de pantalla a un array de numpy
        screenshot_bgr = cv2.cvtColor(screenshot_np, cv2.COLOR_RGB2BGR)                                     # Convierto la imagen de RGB a BGR (formato que usa OpenCV)
        screenshot_hsv = cv2.cvtColor(screenshot_bgr, cv2.COLOR_BGR2HSV)                                    # Convierto las imágenes a HSV
        mask = cv2.inRange(screenshot_hsv, lower_hsv, upper_hsv)                                            # Se aplica la máscara HSV para el rango de colores especificado (Ambos colores de piezas juntas)
        result_hsv = cv2.bitwise_and(screenshot_hsv, screenshot_hsv, mask=mask)                             # Se aplica la máscara a la imagen HSV

        screenshot_prev_np = np.array(screenshot_prev)                                                      # Convierto la captura de pantalla anterior a un array de numpy
        screenshot_prev_bgr = cv2.cvtColor(screenshot_prev_np, cv2.COLOR_RGB2BGR)                           # Convierto la imagen anterior de RGB a BGR (formato que usa OpenCV)
        screenshot_prev_hsv = cv2.cvtColor(screenshot_prev_bgr, cv2.COLOR_BGR2HSV)                          # Convierto la imagen anterior de BGR a HSV
        mask_prev = cv2.inRange(screenshot_prev_hsv, lower_hsv, upper_hsv)                                  # Se aplica la máscara HSV para el rango de colores especificado en la imagen anterior (Ambos colores de piezas juntas)
        result_prev_hsv = cv2.bitwise_and(screenshot_prev_hsv, screenshot_prev_hsv, mask=mask_prev)         # Se aplica la máscara a la imagen HSV anterior

        diff = cv2.absdiff(result_hsv, result_prev_hsv)                                                     # Se calcula la diferencia absoluta entre los frames HSV (imagen actual e imagen anterior)
        diff_sum = np.sum(diff)                                                                             # Se calcula la suma de diferencias absolutas por canal (suma todos los valores en el array diff.Al sumar todos estos valores, obtenemos una medida de la cantidad total de cambio entre los dos fotogramas), Si diff_sum es muy grande indica un gran cambio, pero si es muy pequeño indica un pequeño cambio

        if diff_sum > 1000000:                                                                              # Si la suma de diferencias es mayor que un umbral, procesar la imagen (Ajustar segun sea el caso)
            filename = os.path.join(save_dir, f"screenshot_{capture_count}.png")                            # Se define la ruta donde se guardará la captura de pantalla en HSV, con un nombre diferente en cada iteracion, dado que cambiara el numero de screenshot
            result_bgr = cv2.cvtColor(result_hsv, cv2.COLOR_HSV2BGR)                                        # Se Convierte la imagen de HSV a BGR para guardarla correctamente (con colores y fondo negro)
            cv2.imwrite(filename, result_bgr)                                                               # Se guarda la imagen procesada en el archivo especificado
            print(f"Captura guardada como {filename}")                                                      # Se imprime un mensaje indicando que la captura ha sido guardada
            
            nombre_variable = deteccion(lower_hsv_white[0], lower_hsv_white[1], lower_hsv_white[2], upper_hsv_white[0], upper_hsv_white[1], upper_hsv_white[2], filename)   # Llamar a la función de detección con la ruta de la última imagen capturada, pero unicamente para las piezas blancas
            MatrizTraspuesta, segmento_horizontal, segmento_vertical = Armado_de_nodos_de_las_casillas([(0,int(850*0.4)),(int(850*0.4),int(850*0.4)),(int(850*0.4),0),(0,0)], filename)   # Se pasan las dimensiones en pixeles del cuadrado del tablero para tener los nodos 9x9 (por el redimensionamiento de la imagen multiplico por el porcentaje)
            casilla_actual_blanca = convertir_y_mostrar_resultados(nombre_variable, segmento_horizontal, segmento_vertical)                                                 # Retorna las casillas de las piezas blancas en formato (fila,columna)
            print("casillas obtenidas para las blancas:\n", casilla_actual_blanca,"\n")                                                                                     # Imprimir las casillas obtenidas para las piezas blancas
            
            nombre_variable = deteccion(lower_hsv_black[0], lower_hsv_black[1], lower_hsv_black[2], upper_hsv_black[0], upper_hsv_black[1], upper_hsv_black[2], filename)   # Llamar a las funciones con la ruta de la última imagen capturada , pero unicamente para las piezas negras
            MatrizTraspuesta, segmento_horizontal, segmento_vertical = Armado_de_nodos_de_las_casillas([(0,int(850*0.4)),(int(850*0.4),int(850*0.4)),(int(850*0.4),0),(0,0)], filename)    # Se pasan las dimensiones en pixeles del cuadrado del tablero para tener los nodos 9x9  (por el redimensionamiento de la imagen multiplico por el porcentaje)
            casilla_actual_negra = convertir_y_mostrar_resultados(nombre_variable, segmento_horizontal, segmento_vertical)                                                  # Retorna las casillas de las piezas negras en formato (fila,columna)
            print("casillas obtenidas para las negras:\n", casilla_actual_negra,"\n")                                                                                       # Imprimir las casillas obtenidas para las piezas negras

            origen_blancas, destino_blancas, pieza_comida_blancas,enroque = determinar_cambio_de_piezas(casillas_anterior_blanca, casilla_actual_blanca, casillas_anterior_negra, casilla_actual_negra) # Determino los cambios en las posiciones de las piezas blancas en el tablero

            if origen_blancas is not None and destino_blancas is not None:                                                  # Verifico si se detectó un movimiento válido para las piezas blancas
                tablero_actual, vector_movimiento = mover_pieza(tablero_actual, origen_blancas, destino_blancas,enroque)    # Muevo la pieza blanca del origen al destino en el tablero
                print("Vector de movimiento:", vector_movimiento)                                                           # Imprimo el vector de movimiento de la pieza blanca
                for fila in tablero_actual:                                                                                 # Imprimo el tablero actualizado                                                       
                        print(fila) 
                #enviar_coordenadas_a_arduino(vector_movimiento)                                                            # Se envian las coordenadas de las piezas blancas al arduino.(comentarlo si no queres enviar piezas blancas)

            origen_negras, destino_negras, pieza_comida_negras,enroque = determinar_cambio_de_piezas(casillas_anterior_negra, casilla_actual_negra, casillas_anterior_blanca, casilla_actual_blanca)    # Determino los cambios en las posiciones de las piezas negras en el tablero

            if origen_negras is not None and destino_negras is not None:                                                    # Verifico si se detectó un movimiento válido para las piezas negras
                tablero_actual, vector_movimiento = mover_pieza(tablero_actual, origen_negras, destino_negras,enroque)      # Muevo la pieza negra del origen al destino en el tablero
                print("Vector de movimiento:", vector_movimiento)                                                           # Imprimo el vector de movimiento de la pieza negra
                for fila in tablero_actual:                                                                                 # Imprimo el tablero actualizado                                                           
                        print(fila)
                enviar_coordenadas_a_arduino(vector_movimiento)                                                             # Se envian las coordenadas de las piezas negras al arduino.

            casillas_anterior_blanca = casilla_actual_blanca.copy()                                                         # Actualizo las casillas anteriores de las piezas blancas para la próxima iteración
            casillas_anterior_negra = casilla_actual_negra.copy()                                                           # Actualizo las casillas anteriores de las piezas negras para la próxima iteración
            capture_count += 1                                                                                              # Incremento el contador de capturas para actualizar el nombre de las imagenes
            # time.sleep(0.5)                                                                                               # (Opcinal) Pausar la ejecución durante 0.5 segundos antes de la próxima captura

        screenshot_prev = screenshot                                                                                        # Actualizo el frame anterior
        result_bgr = cv2.cvtColor(result_hsv, cv2.COLOR_HSV2BGR)                                                            # Convierto la imagen de HSV a BGR para mostrarla usando OpenCV
        cv2.imshow('Screenshot', result_bgr)                                                                                # Muestro la captura de pantalla, en la ventana de OpenCV

        if cv2.waitKey(30) & 0xFF == ord('q'):                                                                              # Espero unos milisegundos y termino las capturas de pantalla si se presiona 'q'
            break                                                                                                           # La palabra clave break es para salir de un bucle. En este contexto, break sale del bucle while True que está ejecutando el código de captura de pantalla. Cuando se presiona la tecla 'q', el bucle se detiene

# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#

def main():
    print("Presiona la tecla 'c' para iniciar la captura de pantalla...\n")                                                 # Imprimo un mensaje en la consola para indicarle al usuario que presione la tecla 'c' para iniciar la captura de pantalla
    keyboard.wait('c')                                                                                                      # Espero a que el usuario presione la tecla 'c' antes de continuar con la ejecución del código
    capture_screenshots()                                                                                                   # Llamo a la función capture_screenshots para comenzar a capturar las pantallas
    cv2.destroyAllWindows()                                                                                                 # Cierro todas las ventanas creadas por OpenCV cuando la función capture_screenshots haya terminado


# CODIGO GENERAL PARA USAR EN CHESS.COM

In [2]:

if __name__ == "__main__":                                                                                                  # __name__ es una variable especial en Python que contiene el nombre del módulo actual.Cuando un archivo de Python se ejecuta directamente (por ejemplo, python mi_script.py), el valor de __name__ es '__main__' (Este condicional se usa para determinar si el archivo se está ejecutando directamente y no importado como un módulo.)
    main()                                                                                                                  # Llamo a la función main para iniciar el programa de chess.com


Presiona la tecla 'c' para iniciar la captura de pantalla...

Captura guardada como C:\Users\gasto\OneDrive\Escritorio\Codigos\Python\Codigo de chess.com\Capturas de pantalla\screenshot_0.png

Cantidad de coordenadas: 16
casillas obtenidas para las blancas:
 [(7, 7), (7, 0), (7, 6), (7, 5), (7, 2), (7, 1), (7, 3), (7, 4), (6, 7), (6, 6), (6, 5), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0)] 


Cantidad de coordenadas: 16
casillas obtenidas para las negras:
 [(1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (0, 7), (0, 0), (0, 6), (0, 5), (0, 2), (0, 1), (0, 4), (0, 3)] 

Captura guardada como C:\Users\gasto\OneDrive\Escritorio\Codigos\Python\Codigo de chess.com\Capturas de pantalla\screenshot_1.png

Cantidad de coordenadas: 16
casillas obtenidas para las blancas:
 [(7, 7), (7, 0), (7, 6), (7, 5), (7, 2), (7, 1), (7, 3), (7, 4), (6, 7), (6, 6), (6, 5), (6, 3), (6, 2), (6, 1), (6, 0), (5, 4)] 


Cantidad de coordenadas: 16
casillas obtenidas para las negras:
 [(1, 7), (1, 6), (1